In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

In [2]:
from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

In [ ]:
llm.invoke("What is stock?")

In [62]:
from langchain_core.prompts import ChatPromptTemplate

# llm = ChatOllama(model="llama3:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "What is stock?"})

AIMessage(content='안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점을 도와드리겠습니다. 주식에 대해 설명해 드리겠습니다.\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 예를 들어, 어떤 회사가 100개의 주식을 발행했다면, 각 주식을 구매한 사람은 해당 회사의 1%를 소유하게 됩니다. 이러한 방식으로 사람들은 다양한 사업에 투자하고 수익을 창출할 수 있습니다.\n\n주식은 거래소에서 사고팔 수 있으며, 주식의 가격은 공급과 수요에 따라 변동됩니다. 또한, 기업들은 보통 매년 배당금을 지급하는데, 이는 투자자들에게 주식을 보유하고 있는 대가로 받는 수익입니다.\n\n요약하자면, 주식은 사람들이 회사의 일부를 소유하고 수익을 창출할 수 있게 해주는 금융 상품으로, 거래소에서 사고팔 수 있으며 배당금 형태로 수익을 얻을 수 있습니다.', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-05-06T07:58:57.581603Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 32445901083, 'load_duration': 7018976292, 'prompt_eval_count': 53, 'prompt_eval_duration': 995681000, 'eval_count': 181, 'eval_duration': 24423209000}, id='run-4517d984-140b-44b0-97b4-11c123208d84-0')

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "What is stock?"})

'안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점을 도와드리겠습니다. 주식에 대해 설명해 드리겠습니다.\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 예를 들어, 어떤 회사가 100개의 주식을 발행했다면, 각 주식을 구매한 사람은 해당 회사의 1%를 소유하게 됩니다. 이러한 방식으로 사람들은 다양한 사업에 투자하고 수익을 창출할 수 있습니다.\n\n주식은 거래소에서 사고팔 수 있으며, 주식의 가격은 공급과 수요에 따라 변동됩니다. 또한, 기업들은 보통 매년 배당금을 지급하는데, 이는 투자자들에게 주식을 보유하고 있는 대가로 받는 수익입니다.\n\n요약하자면, 주식은 사람들이 회사의 일부를 소유하고 수익을 창출할 수 있게 해주는 금융 상품으로, 거래소에서 사고팔 수 있으며 배당금 형태로 수익을 얻을 수 있습니다.'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# llm = ChatOllama(model="llama3:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])

output_parser = StrOutputParser()

# chain = prompt | llm | output_parser
my_input = {"input": "What is stock?"}
print(f'1st chain >> {prompt.invoke(my_input)}')
print(f'2nd chain >> {(prompt | llm).invoke(my_input)}')
print(f'3rd chain >> {(prompt | llm | output_parser).invoke(my_input)}')

1st chain >> messages=[SystemMessage(content='You are a helpful, professional assistant named 권봇. answer me in Korean no matter what'), HumanMessage(content='What is stock?')]
2nd chain >> content='주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자들이 회사의 성장과 성공에 기여하고자 자본을 제공함으로써 취득하게 됩니다. 주식을 구매하면 해당 기업의 일부가 되며, 그 성과에 따라 배당금이나 미래 주식 가치 상승으로 인한 이익을 얻게 됩니다. 또한, 주식을 매매하여 가격 변동에 따른 차익을 실현할 수도 있습니다.' response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-05-06T03:09:00.611458Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 16761211959, 'load_duration': 6395765875, 'prompt_eval_count': 40, 'prompt_eval_duration': 979833000, 'eval_count': 79, 'eval_duration': 9379067000} id='run-23955113-0cc5-4ad0-a48a-7accd66da0de-0'
3rd chain >> 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 

In [ ]:
chain = prompt | llm | output_parser
for token in chain.stream(
    {"input": "What is stock?"}
):
    print(token, end="")

주식(stu-gik)은 일반적으로 공 司의 주가를 의미하는 단어입니다. 또한 주식을 매각하거나 매입하여 이익을 얻는 투자 방식을도 나타냅니다. 다른 말로서는 주식 시장에서 거래되는 증권을 총괄하는 말입니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

# 첫번째 체인
prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
chain1 = (
    prompt1 
    | llm 
    | StrOutputParser()
)
message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
print(f'message1: {message1}')

# 두번째 체인 (첫번째 체인 연결)
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])
chain2 = (
    {"input": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)
message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
print(f'message2: {message2}')

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
message = remote_chain.invoke({
    "input": "What is stock?",
})
print(f'message : {message}')

message : 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 그들의 포트폴리오의 다양화와 위험 분산을 가능하게 합니다.


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../assets/news.txt')
data = loader.load()

print(f'type : {type(data)} / len : {len(data)}')
print(f'data : {data}')
print(f'page_content : {data[0].page_content}')

type : <class 'list'>
len : 1
news_data : [Document(page_content="엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n\n이에 따르면 150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.\n\n두 스타트업은 모두 생성 AI 모델 개발이나 실행 비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\n\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으로 공유할 수 있도록 지원하는 쿠버네티스 기반 컨테이너 플랫폼을 개발한다.\n\n특히 GPU 클러스터 리소스를 여러 AI 워크로드에 공유하기 위해 AI '가상화 인프라'를 구축한다는 것이 특징이다. 이를 활용하면 컴퓨팅에 가상 계층을 구축, 적은 리소스를 쓰면서 빠르게 GPU에서 AI 모델을 학습할 수 있다.\n\n데시는 AI 칩에서

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# BeautifulSoup : HTML 및 XML 문서를 파싱하고 구문 분석하는 데 사용되는 파이썬 라이브러리. 주로 웹 스크레이핑(웹 페이지에서 데이터 추출) 작업에서 사용되며, 웹 페이지의 구조를 이해하고 필요한 정보를 추출하는 데 유용
loader = WebBaseLoader(
    web_paths=("https://www.aitimes.com/news/articleView.html?idxno=159102"
               , "https://www.aitimes.com/news/articleView.html?idxno=159072"
               , "https://www.aitimes.com/news/articleView.html?idxno=158943"
               ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "article", # 태그
            attrs={"id": ["article-view-content-div"]}, # 태그의 ID 값들
        )
    ),
)
data = loader.load()

print(f'type : {type(data)} / len : {len(data)}')
print(f'data : {data}')
for d in data:
    print(f'page_content : {d.page_content}')

type : <class 'list'> / len : 3
data : [Document(page_content="\n\n\n(사진=엔비디아)\n\n\n엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n이에 따르면\xa0150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.\n두 스타트업은 모두 생성\xa0AI 모델\xa0개발이나 실행\xa0비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으로 공유할 수 있도록 지원하는 쿠버네티스 기반 컨테이너 플랫폼을 개발한다.\n특히 GPU 클러스터 리소스를 여러 AI 워크로드에 공유하기 위해 AI '가상화 인프라'를 구축한다는 것이 특징이다. 이를 활용하면 컴퓨팅에 가상 계층을 구축, 적은 리소스를 쓰면서 빠르게 GPU에서 AI 모델을 학습할 수 있

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf")
pages = loader.load()

print(f'type : {type(pages)} / len : {len(pages)} / pages : {pages}')

type : <class 'list'> / len : 227 / pages : [Document(page_content=' \n연구책임자 : 김 주 희(국 립 부 경 대 학 교)\n공동연구자 :차 재 권(국 립 부 경 대 학 교)\n김 현 정(동 아 대 학 교)\n조 성 복(국 민 대 학 교)\n연구보조원 : 박 서 현(국 립 부 경 대 학 교)\n권 수 민(국 립 부 경 대 학 교)\n국립부경대학교 산학협력단2023년도 중앙선거관리위원회 정책연구용역 보고서\n생성형 AI 신기술 도입에 따른 \n선거 규제 연구', metadata={'source': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'page': 0}), Document(page_content='ii【2023년도 중앙선거관리위원회 정책연구용역 보고서】\n『생성형 AI 신기술 도입에 따른 \n선거 규제 연구』\n연구책임자 : 김 주 희(국 립 부 경 대 학 교)\n공동연구자 :차 재 권(국 립 부 경 대 학 교)\n김 현 정(동 아 대 학 교)\n조 성 복(국 민 대 학 교)\n연구보조원 : 박 서 현(국 립 부 경 대 학 교)\n권 수 민(국 립 부 경 대 학 교)\n본 연구보고서는 2023년도 중앙선거관리위원회 정책연구용역 과제로서 연구내용은 중앙\n선거관리위원회의 공식 견해가 아님.', metadata={'source': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'page': 1}), Document(page_content='iii차 례\n1. 서론 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(data)
print(f'len(splits[0].page_content) : {len(splits[1].page_content)}')
splits

len(splits[0].page_content) : 456


[Document(page_content='(사진=엔비디아)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159102'}),
 Document(page_content='엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n이에 따르면\xa0150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159102'}),
 Document(page_content="두 스타트업은 모두 생성\xa0AI 모델\xa0개발이나 실행\xa0비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")
embed = embeddings.embed_documents(
    [
        "안녕 영광",
        "동해물과 백두산",
        "마르고 닳도록",
        "하느님이 보우하사",
        "우리나라 만세"
    ]
)
len(embed), len(embed[0]), len(embed[1]), embed

(5,
 4096,
 4096,
 [[0.5027266144752502,
   -2.5857994556427,
   1.7093673944473267,
   1.093870997428894,
   4.407754898071289,
   1.8732117414474487,
   5.734858989715576,
   -3.461538553237915,
   -0.8753273487091064,
   -7.282148838043213,
   -0.9676973819732666,
   -0.7342850565910339,
   -1.471280813217163,
   -4.048448085784912,
   0.9023484587669373,
   3.671241044998169,
   6.189359664916992,
   5.910055160522461,
   -3.200493812561035,
   0.14421378076076508,
   1.7674579620361328,
   -4.080934524536133,
   5.155828475952148,
   0.4132400155067444,
   -8.15284252166748,
   7.216220855712891,
   -0.9999527931213379,
   5.744194030761719,
   -6.457835674285889,
   2.5026087760925293,
   1.4653778076171875,
   2.2383477687835693,
   4.365627765655518,
   -3.2420098781585693,
   -3.2670254707336426,
   0.3542901873588562,
   4.901410102844238,
   7.789434909820557,
   -1.7037307024002075,
   5.0356621742248535,
   2.402841567993164,
   -0.4054710268974304,
   -5.1353535652160645,

In [89]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

vectorstore = FAISS.from_documents(splits,
                                   embedding = OllamaEmbeddings(model="EEVE-Korean-Instruct-10.8B-v1.0:latest"),
                                   distance_strategy = DistanceStrategy.COSINE # 코사인 유사도 측정. 값이 클수록 더 유사함을 의미
                                  )

# 로컬에 DB 저장
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore.save_local(MY_FAISS_INDEX)

vectorstore

In [4]:
# 로컬 DB 불러오기
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore = FAISS.load_local(MY_FAISS_INDEX, OllamaEmbeddings(model="EEVE-Korean-Instruct-10.8B-v1.0:latest"), allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출
retrieved_docs = retriever.invoke("라마3")
retrieved_docs

[Document(page_content='(사진=메타)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='(사진=엔비디아)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159102'}),
 Document(page_content="라마 3 벤치마크 결과 (사진=메타)\n\n\n라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에는 강하지만, 70B의 경우 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 떨어졌다.\xa0\n특히 인간 선호도에서 경쟁 모델을 앞서는 것으로 알려졌다.\n조언 요청, 브레인스토밍, 분류, 비공개 질문 답변, 코딩, 창의적인 글쓰기, 추출, 공개 질문 답변, 추론, 재작성 및 요약 등 12가지 주요 사용 사례를 포함한 1800개\xa0프롬프트\xa0구축\xa0데이터셋에 대한 인간 평가에서 오픈AI의 'GPT-3.5', 미스트랄 7B, 클로드 3 소네트보다 높게 평가됐다.\n\n\n라마 3 인간 평가 결과 (사진=메타)", metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='퍼플렉시티 공동창립자 조니 호, 아라빈드 스리니바스, 데니스 야라츠 (사진=퍼플렉시티)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159072'}),
 Document(page_content='라마 3 인간 평가 결과 (사진=메타)\n\n\n허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드\xa01위에 오르며 역대 가장 빠른

In [5]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt") # https://smith.langchain.com/hub/rlm/rag-prompt

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
chain.invoke('퍼플렉시티가 투자받은 금액?')

'퍼플렉시티는 최근 약 6300만 달러(약 860억 원)의 투자를 유치했으며, 이로 인해 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가되었습니다.'